In [43]:
import numpy as np 
import pandas as pd
import vrplib
import collections
from scipy.cluster.hierarchy import linkage, fcluster,inconsistent
from scipy.spatial.distance import pdist

In [44]:
# read VRPLIB formatted instances (default)

instance = vrplib.read_instance("C:/Users/cevat/Desktop/assignment/CVRP-Heuristics-Lab/Vrp-Set-D/Vrp-Set-D/D/ORTEC-n242-k12.vrp")
solution = vrplib.read_solution("C:/Users/cevat/Desktop/assignment/CVRP-Heuristics-Lab/Vrp-Set-D/Vrp-Set-D/D/ORTEC-n242-k12.sol")
instance.keys()

dict_keys(['name', 'comment', 'type', 'dimension', 'edge_weight_type', 'edge_weight_format', 'node_coord_type', 'capacity', 'edge_weight', 'node_coord', 'demand', 'depot'])

In [45]:
def hierarchical_clustering(instance):

    vehicle_capacity = instance["capacity"]
    number_of_nodes = len(instance["node_coord"]) 



    # calculate customers vs depot coordinate differences.
    customers = pd.DataFrame(columns=["Demand","DistanceToDepot","CoordDifferenceX","CoordDifferenceY"])
    for customer in range(1,number_of_nodes):
        coord_diff_x = instance["node_coord"][customer,0] - instance["node_coord"][0,0]
        coord_diff_y = instance["node_coord"][customer,1] - instance["node_coord"][0,1]
        customers.loc[customer, "DistanceToDepot"] = instance["edge_weight"][customer,0]
        customers.loc[customer, "Demand"] = instance["demand"][customer]
        customers.loc[customer, "CoordDifferenceX"] = coord_diff_x
        customers.loc[customer, "CoordDifferenceY"] = coord_diff_y


    # Perform hierarchical clustering on coordinate differences
    coords = customers[["CoordDifferenceX", "CoordDifferenceY"]].values
    distances = pdist(coords)  # Calculate the pairwise distances
    linkage_matrix = linkage(distances, method='ward')  # Perform hierarchical clustering

    # Calculate the inconsistency matrix
    inconsistency_matrix = inconsistent(linkage_matrix)

    max_inconsistency = np.max(inconsistency_matrix[:, 0])
    threshold = max_inconsistency * 0.5 

    # Assign clusters based on the dynamic threshold
    customers["Cluster"] = fcluster(linkage_matrix, threshold, criterion='distance')

    # Sort customers by Cluster, then Demand (descending), DistanceToDepot (ascending)
    sorted_customers = customers.sort_values(by=["Cluster", "DistanceToDepot", "Demand"], ascending=[True, True, False])


    # create initial solution dict

    routes = collections.defaultdict(list)

    candidate_vehicle_cap = vehicle_capacity
    candidate_vehicle_index = 0 
    for i in sorted_customers.index: 
        if candidate_vehicle_cap >= sorted_customers.loc[i,"Demand"]   : 
            routes[candidate_vehicle_index].append(i) 
            candidate_vehicle_cap -= sorted_customers.loc[i,"Demand"]
        else:
            candidate_vehicle_index += 1
            routes[candidate_vehicle_index].append(i)
            candidate_vehicle_cap = vehicle_capacity
            candidate_vehicle_cap -= sorted_customers.loc[i,"Demand"]
    return routes

In [46]:

def nearest_neighbor(instance):
    n = len(instance["demand"])
    visited = np.zeros(n, dtype=bool)  # Track visited nodes
    routes = {}
    route_index = 0
    depot = 0  # Assuming the depot is at index 0
    
    while not all(visited[1:]):  # Continue until all customers are visited
        current_route = [depot]  # Start a new route from the depot
        current_load = 0
        
        while True:
            last_node = current_route[-1]
            nearest_node = None
            nearest_distance = float('inf')
            
            # Find the nearest unvisited customer that can be added without exceeding capacity
            for i in range(1, n):
                if not visited[i] and instance["demand"][i] + current_load <= instance["capacity"]:
                    distance = instance["edge_weight"][last_node][i]
                    if distance < nearest_distance:
                        nearest_distance = distance
                        nearest_node = i
            
            if nearest_node is None:  # No more customers can be added to this route
                break
            
            # Add the nearest customer to the route
            current_route.append(nearest_node)
            current_load += instance["demand"][nearest_node]
            visited[nearest_node] = True
        
        current_route.append(depot)  # Return to the depot to complete the route
        routes[route_index] = current_route  # Store the route in the dictionary
        route_index += 1
    
    return routes



In [47]:
import heapq

def clarke_wright_savings(instance):
    n = len(instance["demand"])
    savings = []
    # Initialize each customer in a separate route
    routes = {i: [i] for i in range(1, n)}
    capacities = {i: instance["demand"][i] for i in range(1, n)}
    
    # Calculate savings for merging each pair of customers
    for i in range(1, n):
        for j in range(i+1, n):
            save = instance["edge_weight"][0][i] + instance["edge_weight"][0][j] - instance["edge_weight"][i][j]
            heapq.heappush(savings, (-save, i, j))
    
    while savings:
        save, i, j = heapq.heappop(savings)
        route_i = next((route for route in routes.values() if route[0] == i), None)
        route_j = next((route for route in routes.values() if route[-1] == j), None)
        
        # Merge routes if they are distinct and the merge does not exceed vehicle capacity
        if route_i and route_j and route_i != route_j:
            if capacities[route_i[0]] + capacities[route_j[-1]] <= instance["capacity"]:
                # Merge route_j into route_i
                route_i.extend(route_j)
                capacities[route_i[0]] += capacities[route_j[-1]]
                del routes[route_j[0]]
    
    # Reformat routes to be indexed by route index
    routes_final = {index: route for index, route in enumerate(routes.values())}
    
    return routes_final



In [48]:
def calculate_costs(solution,instance,penalty_per_unit):
    total_weight_cost = 0 
    total_weight_cost_with_penalty = 0
    route_cost = collections.Counter()
    total_demand = collections.Counter()
    for vehicle,route in solution.items():
        for node in route:
            total_demand[vehicle] += instance["demand"][node]

        for node in range(len(route)-1):
            route_cost[vehicle] += instance["edge_weight"][node,node+1]

        total_weight_cost += route_cost[vehicle]
        total_weight_cost_with_penalty += route_cost[vehicle]

        if total_demand[vehicle] > instance["capacity"] :
            overcapacity = total_demand[vehicle] - instance["capacity"]
            total_weight_cost_with_penalty += penalty_per_unit * overcapacity
    
    return total_weight_cost,total_weight_cost_with_penalty,dict(route_cost),dict(total_demand)




In [49]:
sol1= hierarchical_clustering(instance)
sol2 = nearest_neighbor(instance)
sol3 = clarke_wright_savings(instance)
calculate_costs(sol1,instance,100)

(493832.0,
 493832.0,
 {0: 41366.0,
  1: 41783.0,
  2: 40518.0,
  3: 40518.0,
  4: 40518.0,
  5: 42587.0,
  6: 42587.0,
  7: 41366.0,
  8: 37701.0,
  9: 41783.0,
  10: 40518.0,
  11: 42587.0},
 {0: 122,
  1: 123,
  2: 124,
  3: 124,
  4: 123,
  5: 124,
  6: 122,
  7: 122,
  8: 123,
  9: 124,
  10: 119,
  11: 121})

In [53]:
calculate_costs(sol3,instance,100)

(437081.0, 571681.0, {0: 437081.0}, {0: 1471})